In [ ]:
import xgboost as xgb
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt  
from pdpbox import info_plots
from xgboost.sklearn import XGBClassifier
from pdpbox import pdp
from matplotlib import cm
from sklearn.metrics import roc_auc_score
from geopy import distance
from time import time
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
 
plt.rc('font',family='Arial')
plt.rcParams ['svg.fonttype'] ='none'


### West Region

In [ ]:
path = r'../../data/Different-ALAN/'
os.chdir(path)
data=pd.read_excel('west.xlsx')
data['month'] = data['date'].apply(lambda x:pd.Timestamp(x).month)
data_base = data[['type','2 metre temperature', 'Total precipitation', 'Barren',
       'Cultivated and Managed Vegetation', 'Deciduous Broadleaf Trees',
       'Evergreen Broadleaf Trees', 'Evergreen Deciduous Needleleaf Trees',
       'Herbaceous Vegetation', 'Mixed-Other Trees',
       'Regularly Flooded Vegetation', 'Shrubs', 
       'Pop', 'year_precipitation', 'elevation', 'NDVI', 
       'distance_water', '10 metre U wind component',
       '10 metre V wind component', 'month']]
X= data_base.iloc[:,1:]
y = data_base.iloc[:,0]
data_light = data[[ 'distance_light0.3', 'distance_light0.5', 'distance_light1',
       'distance_light2', 'distance_light3.5', 'distance_light5',
       'distance_light9.1', 'distance_light17']]#, 'distance_light37'
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, random_state=0)
light_train, light_test = train_test_split(data_light, test_size=0.4, random_state=0)

In [ ]:
pdp_df_list = []
my_bins = np.arange(0,401,20)
#pdp_df_list.append(pd.Series(my_bins))
for i in light_train.columns:
    print(i)
    X_train_1 = pd.concat([X_train,light_train[i]],axis=1)
    X_test_1 = pd.concat([X_test,light_test[i]],axis=1)
    '''xgb_model = XGBClassifier(
                      n_jobs=4,
                      max_depth=6,
                      learning_rate=0.1,
                      n_estimators=300,
                      verbosity=1,
                      objective='binary:logistic',
                      booster='gbtree',
                      gamma=1,
                      min_child_weight=4,
                      subsample=0.6,
                      colsample_bytree=0.6,
                      reg_alpha=1,
                      reg_lambda=10,
                      base_score=0.5,
                      eval_metric='auc',
                  )'''
    xgb_model = XGBClassifier(
          n_jobs=4,
          max_depth=8,
          learning_rate=0.03,
          n_estimators=300,
          verbosity=1,
          objective='binary:logistic',
          booster='gbtree',
          gamma=0.4,
          min_child_weight=4,
          subsample=0.9,
          colsample_bytree=0.8,
          reg_alpha=0.1,
          reg_lambda=1,
          base_score=0.5,
          eval_metric='auc',
          )
    xgb_model.fit(X_train_1, y_train,eval_set=[(X_test_1, y_test)],verbose=False)
    data_features = list(X_train_1.columns)

    pdp_light = pdp.pdp_isolate(xgb_model,X_train_1,data_features,i,cust_grid_points=my_bins)
    pdp_df = pd.DataFrame([my_bins,pdp_light.pdp,[i]*len(my_bins)]).T
    pdp_df.columns=['variable','pdp','v']
    pdp_df_list.append(pdp_df)
    #break

In [ ]:
c1 = pd.concat(pdp_df_list,axis=0)
c1.variable=c1.variable.astype(int)
c1.pdp=c1.pdp.astype(float)
sns.set(style='white', font='sans-serif',font_scale=2)
ax = sns.lineplot(x='variable', y='pdp',palette='Reds', hue='v',data=c1)#Reds,Blues
plt.legend(fontsize=15,loc=(1,0),frameon=False)
plt.title('Western China',fontsize=20)
ax.set_ylim([0.45,0.60])
ax.set_xlabel(xlabel='Distance to light (km)',fontdict={'size':20})   
ax.set_ylabel(ylabel='Distribution probability',fontdict={'size':20}) 
plt.savefig(r'../../result/figure/Different-ALAN/west.png',dpi=300,bbox_inches = 'tight')

### East Region

In [ ]:
path = r'../../data/Different-ALAN/'
os.chdir(path)
data=pd.read_excel('east.xlsx')
data['month'] = data['date'].apply(lambda x:pd.Timestamp(x).month)
data_base = data[['type','2 metre temperature', 'Total precipitation', 'Barren',
       'Cultivated and Managed Vegetation', 'Deciduous Broadleaf Trees',
       'Evergreen Broadleaf Trees', 'Evergreen Deciduous Needleleaf Trees',
       'Herbaceous Vegetation', 'Mixed-Other Trees',
       'Regularly Flooded Vegetation', 'Shrubs', 
       'Pop', 'year_precipitation', 'elevation', 'NDVI', 
       'distance_water', '10 metre U wind component',
       '10 metre V wind component', 'month']]
X= data_base.iloc[:,1:]
y = data_base.iloc[:,0]
data_light = data[[ 'distance_light0.3', 'distance_light0.5', 'distance_light1',
       'distance_light2', 'distance_light3.5', 'distance_light5',
       'distance_light9.1', 'distance_light17']]#, 'distance_light37'
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, random_state=0)
light_train, light_test = train_test_split(data_light, test_size=0.4, random_state=0)

In [ ]:
pdp_df_list = []
my_bins = np.arange(0,401,20)
#pdp_df_list.append(pd.Series(my_bins))
for i in light_train.columns:
    print(i)
    X_train_1 = pd.concat([X_train,light_train[i]],axis=1)
    X_test_1 = pd.concat([X_test,light_test[i]],axis=1)
    xgb_model = XGBClassifier(
                      n_jobs=4,
                      max_depth=6,
                      learning_rate=0.1,
                      n_estimators=300,
                      verbosity=1,
                      objective='binary:logistic',
                      booster='gbtree',
                      gamma=1,
                      min_child_weight=4,
                      subsample=0.6,
                      colsample_bytree=0.6,
                      reg_alpha=1,
                      reg_lambda=10,
                      base_score=0.5,
                      eval_metric='auc',
                  )
    xgb_model.fit(X_train_1, y_train,eval_set=[(X_test_1, y_test)],verbose=False)
    data_features = list(X_train_1.columns)

    pdp_light = pdp.pdp_isolate(xgb_model,X_train_1,data_features,i,cust_grid_points=my_bins)
    pdp_df = pd.DataFrame([my_bins,pdp_light.pdp,[i]*len(my_bins)]).T
    pdp_df.columns=['variable','pdp','v']
    pdp_df_list.append(pdp_df)
    #break

In [ ]:
c1 = pd.concat(pdp_df_list,axis=0)
c1.variable=c1.variable.astype(int)
c1.pdp=c1.pdp.astype(float)
sns.set(style='white', font='sans-serif',font_scale=2)
ax = sns.lineplot(x='variable', y='pdp',palette='Blues', hue='v',data=c1)#Reds,Blues
plt.legend(fontsize=15,loc=(1,0),frameon=False)
plt.title('Eastern China',fontsize=20)
ax.set_ylim([0.45,0.60])
ax.set_xlabel(xlabel='Distance to light (km)',fontdict={'size':20})   
ax.set_ylabel(ylabel='Distribution probability',fontdict={'size':20}) 
plt.savefig(r'../../result/figure/Different-ALAN/east.png',dpi=300,bbox_inches = 'tight')